In [1]:
!which python

/home/charx/python-virtual-environments/stonks-env/bin/python


In [2]:
from stonks.data.test_imports import say_hello
from pandas_datareader import data


# vanilla markotwitz + l2 regularized
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import objective_functions 


# black litterman model
from pypfopt import black_litterman
from pypfopt.black_litterman import BlackLittermanModel
from pypfopt.efficient_frontier import EfficientFrontier


import yfinance as yf
import pandas as pd
import numpy as np
import time
import datetime as datetime

In [3]:
tickers = [
    'AAPL', # Apple -> tech
    'TSLA', # Tesla -> tech
    'GOOGL', # Google -> tech
    'AMZN', # Amazon -> consumer/tech
    'MA', # MasterCard -> finance
    'V', # Visa -> finance
    'MELI', # MercadoLibre -> consumer
    'NVDA', # Nvidia -> chips
    'ASML', # ASML -> chips
    'FB' # facebook -> social media
]

In [5]:
# We would like all available data from 01/01/2000 until 12/31/2016.
start_date = '2008-01-01'
end_date = '2021-04-05'

# possible bug correction
# start_date = '01-01-2010'
# end_date = '03-07-2021'
# start = datetime.datetime.strptime(start_date, '%d-%m-%Y')
# end = datetime.datetime.strptime(end_date, '%d-%m-%Y')

In [22]:
# single ticker dload doesnt work!!
# curr_df = data.DataReader('GOOG', 'yahoo', start, end) # test the yf downloader

In [10]:
# legacy version to read data
# price_data = []
# for ticker in tickers:
#     print(ticker)
#     curr_df = data.DataReader(ticker, 'yahoo', start_date, end_date)
#     time.sleep(0.3)
#     price_data.append(curr_df['Adj Close'])

# # construct a df with the company values
# df_stocks = pd.concat(price_data, axis = 1)
# df_stocks.columns = tickers

tmp = pd.DataFrame(yf.download(tickers,start=start_date,end=end_date))
df_stocks = tmp['Adj Close']
df_stocks = df_stocks.fillna(method='bfill')

[*********************100%***********************]  10 of 10 completed


In [11]:
# LEGACY -> using pandas bfill method now
def back_fill_df(df, verbose = True):
    df_cp = df.copy()
    
    tkrs = list(df.columns)
    
    for tiker in tkrs:
        print(f'Backfilling: {tiker}') if verbose else 'continue'
        #srs = df_cp[df_cp[tiker].isnull()][tiker]
        srs = df_cp[~df_cp[tiker].isnull()][tiker]

        # only do the backfill if the series requires it
        if srs.shape[0] > 0:
            bkfill_val = srs[0]
            
            print(f"Backfilldate {srs.index[0]}") if verbose else 'continue'
            #bkfill_val = df_cp[df_cp.index == last_nan + pd.DateOffset(1)][ticker][0]
            
            print(f"Backfilling with {bkfill_val}") if verbose else 'continue'
            df_cp[tiker] = df_cp[tiker].fillna(value = bkfill_val)
    
    return df_cp

In [12]:
# LEGACY backfill method
#df_stocks_bkfilled = back_fill_df(df_stocks, verbose = True)

In [32]:
df_stocks_bkfilled = df_stocks
df_stocks_bkfilled.shape
#df_stocks_bkfilled = 12

# try:
#     assert df_stocks_bkfilled.empty == False
# except AssertionError:
#     print("The df is empty :(")


(3337, 10)

In [33]:
# get mean and cover matrices
mu = mean_historical_return(df_stocks_bkfilled)
S = CovarianceShrinkage(df_stocks_bkfilled).ledoit_wolf()

# with l2 regularization to not have 0s in the weights

ef = EfficientFrontier(mu, S)
ef.add_objective(objective_functions.L2_reg, gamma=0.5)
w = ef.max_sharpe()

cleaned_weights = ef.clean_weights()
ef.save_weights_to_file("weights.txt")  # saves to file
print(cleaned_weights)
ef.portfolio_performance(verbose=True)

OrderedDict([('AAPL', 0.11133), ('AMZN', 0.14094), ('ASML', 0.12624), ('FB', 0.07028), ('GOOGL', 0.01964), ('MA', 0.09401), ('MELI', 0.0556), ('NVDA', 0.05144), ('TSLA', 0.22766), ('V', 0.10285)])
Expected annual return: 30.0%
Annual volatility: 26.4%
Sharpe Ratio: 1.06


/home/charx/python-virtual-environments/stonks-env/lib/python3.7/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:248: UserWarning: max_sharpe transforms the optimization problem so additional objectives may not work as expected.
  "max_sharpe transforms the optimization problem so additional objectives may not work as expected."


(0.2998621747002599, 0.2636093550703512, 1.0616549425022157)

# Black litterman

In [36]:
mcaps = {}
for t in tickers:
    stock = yf.Ticker(t)
    mcaps[t] = stock.info["marketCap"]


In [39]:
mcaps

{'AAPL': 2151114997760,
 'TSLA': 649101508608,
 'GOOGL': 1572586389504,
 'AMZN': 1657491161088,
 'MA': 369596104704,
 'V': 506987249664,
 'MELI': 77152419840,
 'NVDA': 374639132672,
 'ASML': 264838168576,
 'FB': 898004353024}

In [40]:
# get the market prices of the sp500
sp500 = yf.download("SPY", period="max")["Adj Close"]

# declare our views of the market
viewdict = {
    "GOOGL": 0.30,
    "FB": 0.30,
    "ASML": 0.30,
    "NVDA": 0.30,
}

# how confident we are on our views
confidences = [
    0.8,
    0.8,
    0.6,
    0.6,
]
# confidences = [
#     0.3,
#     0.8,
#     0.8,
#     0.7,
#     0.8,
#     0.5,
#     0.6,
#     0.7,
#     0.8,
#     0.9,
#     0.9,
# ]


S = CovarianceShrinkage(df_stocks_bkfilled).ledoit_wolf()
delta = black_litterman.market_implied_risk_aversion(sp500)

# get market priors
market_prior = black_litterman.market_implied_prior_returns(mcaps, delta, S)

# blmodel
bl = BlackLittermanModel(S, pi = market_prior, absolute_views = viewdict, risk_aversion = delta, omega="idzorek", view_confidences=confidences)
#bl = BlackLittermanModel(S, pi = market_prior, risk_aversion = delta, omega="idzorek")


ret_bl = bl.bl_returns() # mean matrix for black litterman
S_bl = bl.bl_cov() # co-variance matrix for black litterman


# then we allocate the portfolio using markowitz regularized
ef = EfficientFrontier(ret_bl, S_bl)
#ef.add_objective(objective_functions.L2_reg, gamma = 0.5)
ef.add_objective(objective_functions.L2_reg)

ef.max_sharpe()
weights = ef.clean_weights()
ef.portfolio_performance(verbose=True)

[*********************100%***********************]  1 of 1 completed
Expected annual return: 28.2%
Annual volatility: 25.8%
Sharpe Ratio: 1.01


/home/charx/python-virtual-environments/stonks-env/lib/python3.7/site-packages/pypfopt/efficient_frontier/efficient_frontier.py:248: UserWarning: max_sharpe transforms the optimization problem so additional objectives may not work as expected.
  "max_sharpe transforms the optimization problem so additional objectives may not work as expected."


(0.2820280065130128, 0.25831918328018505, 1.0143575215194334)

In [94]:
capital = 2401.79
capital = capital - 14.28

total = 0
for k, v in weights.items():
    total += capital * v
    print(f"{k} pct: {v} amnt: {capital * v}")


AAPL pct: 0.10369 amnt: 247.56091189999998
TSLA pct: 0.08624 amnt: 205.89886239999998
GOOGL pct: 0.11307 amnt: 269.9557557
AMZN pct: 0.1167 amnt: 278.622417
MA pct: 0.08609 amnt: 205.5407359
V pct: 0.07667 amnt: 183.05039169999998
MELI pct: 0.09993 amnt: 238.5838743
NVDA pct: 0.10203 amnt: 243.59764529999995
ASML pct: 0.09479 amnt: 226.31207289999998
FB pct: 0.12079 amnt: 288.38733289999993


In [93]:
capital = 1809.92
capital = capital - 14.28

total = 0
for k, v in weights.items():
    total += capital * v
    print(f"{k} pct: {v} amnt: {capital * v}")

AAPL pct: 0.10369 amnt: 186.18991160000002
TSLA pct: 0.08624 amnt: 154.8559936
GOOGL pct: 0.11307 amnt: 203.03301480000002
AMZN pct: 0.1167 amnt: 209.551188
MA pct: 0.08609 amnt: 154.58664760000002
V pct: 0.07667 amnt: 137.6717188
MELI pct: 0.09993 amnt: 179.43830520000003
NVDA pct: 0.10203 amnt: 183.2091492
ASML pct: 0.09479 amnt: 170.2087156
FB pct: 0.12079 amnt: 216.89535560000002


# VaR

In [83]:
#Backfilldate 2019-09-19 00:00:00
# new backfill date 2012-05-18 00:00:00

In [84]:
df = df_stocks_bkfilled.copy()
#df_var = df[df.index > '2019-09-19']
df_var = df[df.index > '2012-05-18']

# get the % vals of the stocks
weights_lst = list(weights.values())
weights_array = np.array(weights_lst)

In [85]:
#sh_wt
#historic_var

In [86]:
df_var = df_var.pct_change().iloc[1:,:]

In [87]:
#PnL = (weights * returns.values).sum(axis=1)
# calculate the daily historic PnL of the portfolio
hist_daily_portfolio_PnL = (weights_array * df_var.values).sum(axis = 1)

In [88]:
historic_var = np.percentile(hist_daily_portfolio_PnL, 5, interpolation="lower")
print(f"The historic VaR is: {historic_var}")

capital_pandors = 598
capital_puerquits = 4768.50

print(f'The simple historical VaR pandors is {historic_var * capital_pandors}')
print(f'The simple historical VaR puerquits is {historic_var * capital_puerquits}')


The historic VaR is: -0.023620926415302848
The simple historical VaR pandors is -14.125313996351103
The simple historical VaR puerquits is -112.63638761137163


# CVAR

In [89]:
# filter all values with loses greater than the historic var
cvar = hist_daily_portfolio_PnL[hist_daily_portfolio_PnL <= historic_var]
print(f'The cvar of the portfolio is: {cvar.mean()}')
print(f'The simple historical CVaR pandors is {cvar.mean() * capital_pandors}')
print(f'The simple historical VaR puerquits is {cvar.mean() * capital_puerquits}')

The cvar of the portfolio is: -0.035916581778095306
The simple historical CVaR pandors is -21.478115903300992
The simple historical VaR puerquits is -171.26822020884748
